Кобзарь О.С. Хабибуллин Р.А. 2019 г.

# Модель фонтанирующей скважины

In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uWell.deviation_survey as dev_sev
import uniflocpy.uTools.data_workflow as utool
import uniflocpy.uTools.uconst as uconst
import uniflocpy.uWell.uPipe as Pipe
import uniflocpy.uWell.Self_flow_well as self_flow_well
import plotly.graph_objs as go
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import pandas as pd
init_notebook_mode(connected=True)
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
data = utool.Data()
from uniflocpy.uTools import plotly_workflow
import re
import uniflocpy.uPVT.BlackOil_model as BlackOil_model
import uniflocpy.uTemperature as uTemperature
import uniflocpy.uReservoir.IPR_simple_line as IPR_simple_line
import uniflocpy.uTools.plotly_workflow as plotly_workflow 
import uniflocpy.uValidation.python_api as python_api
import uniflocpy.uValidation.by_UniflocVBA as bvba

## Настройки расчета

In [ ]:
calc_options ={"step_lenth_in_calc_along_wellbore_m":25,
                "without_annulus_space":False,
                "solver_using":True}

## Данные по скважинам

Скв. 2Ф Исследование 5РМ

In [ ]:
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
 "gamma_gas" : 1.45  * 24.05 / 28.98,
'pb_bar': 90}

well_data = {"h_intake_mes_m" :1205.5,
"h_intake_vert_m" : 1205.5,
"h_bottomhole_mes_m" : 1605, 
"h_bottomhole_vert_m": 1605,
             
"geothermal_grad_cm":0.02,
"t_wellhead_c":20,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar': 114.35, 
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":40,
"p_reservoir_bar":177,
"fw_on_surface_perc":0}
real_measurements = pd.DataFrame({'p_survey_mpa': [0.9, 1.12, 1.83, 2.957, 4.355, 5.785, 7.3, 8.953, 9.863, 10.176, 11.435],
                                 'h_mes_survey_m': [0, 105, 305, 505, 705, 905, 1105, 1305, 1405, 1505, 1605]})

## Настройка модели скважины и расчет

Создадим объект - скважину

In [ ]:

simple_well = self_flow_well.self_flow_well(fluid=1, reservoir = 0, pipe=0, temp_corr=1, **fluid_data,
                                                **well_data, **calc_options, activate_rus_mode = 0)
    

Настроим модель

In [ ]:
simple_well.pipe.hydr_cor.pains_corr_using =0
simple_well.pipe.hydr_cor.gravity_grad_coef =0.985
simple_well.pipe.hydr_cor.friction_grad_coef =1

In [ ]:
blackoil_option = BlackOil_model.BlackOil_option()
blackoil_option.set_vba_preset()
simple_well.pipe.fluid_flow.fl.option = blackoil_option

simple_well.pipe.fluid_flow.calc_with_temp_cor = 0 #0 - для линейного распределения температуры,
                                                    #для Hasan и Kabir нужно, ставить 1
                                                    #0 - ускоряет расчет

Запустим расчет снизу вверху (система скважина + пласт)

In [ ]:
simple_well.calc_all_from_down_to_up()

In [ ]:
#simple_well.calc_all_from_up_to_down() можно рассчитывать и сверху-вниз

## Выгрузка расчитанных значений из модели скважины и объединение с фактическими данными

Создадим 2 DataFrame, один с индексов - измеренная глубина, другой - давление

In [ ]:
result_df = simple_well.data.get_data_as_df()
result_df = result_df.set_index('well.h_calculated_mes_m')

In [ ]:
result_df_p = simple_well.data.get_data_as_df()
result_df_p = result_df_p.set_index('well.p_calculated_bar')

Выгрузка ограниченного набора данных в файл Excel

In [ ]:
group_patterns = [['p_calculated_bar'], ['t_calculated'],['gas_fraction'], ['rhon'], ['mun'], ['density'], 
                  ['friction_grad'], ['number_re'], ['pb_bar'], ['fluid_flow', 'msec'], ['vsl'], ['vsg'], ['result_grad'], 
                  ['acceleration'], ['rsb_m3t'], ['gas_liberated_m3t'], ['gas_dissolved_m3t'],
                 ['b_oil_m3m3'], ['rho_oil_kgm3'], ['mu_oil_cp'],['fl.z'], ['rho_gas_dissolved_relative_d']]
antipatterns = ['temp_cor', 'dvdp', 'grad_coef', 'module_friction', 'grad_using', 'for_rus_cor']
print_all = True
columns_to_plot = plotly_workflow.create_columns_to_plot(result_df, group_patterns, antipatterns, print_all)
short_df = result_df[columns_to_plot]
short_df = utool.rename_columns_by_dict(short_df)

class_names = ['hydr_cor.', 'temp_cor.', 'fl.', 'fluid_flow.', 'well.', 'earth']
for i in class_names:
    short_df = utool.rename_columns_by_dict(short_df,{'':i})
short_df.index.name = 'Измеренная глубина, м'
short_df.to_excel('uvalidation_self_flow_well_short_result.xlsx')

Добавление результатов фактических исследований

In [ ]:
real_measurements["p_survey_bar"] = real_measurements["p_survey_mpa"] * 10
real_measurements = real_measurements.set_index(real_measurements['h_mes_survey_m'])
real_measurements.index.name = 'well.h_calculated_mes_m'

result_df = result_df.join(real_measurements, how = 'outer')

Расчет относительной ошибки по давлению на устьес скважины

In [ ]:
(real_measurements["p_survey_bar"][0] - simple_well.p_wellhead_bar) / real_measurements["p_survey_bar"][0] * 100

## Построение графиков

In [ ]:
group_patterns = [["p_bar", 'fl'], ['t_c', 'fl'], ['well.t_calculated_earth_init'], ['survey'], ['p_calculated_bar_vba'], ['t_calculated_c_vba']]
antipatterns = ['mu', 'h_mes_survey', 'mpa', 'fl.']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = [["p_calculated_bar"], ['p_survey']]
antipatterns = ['mu', 'h_mes_survey', 'mpa','fl','ipr']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='КРД',
                                 print_all=False,
                                x_name='Давление, бар',
    y_name='Глубина, м',
    traces_names=['Расчет КРД по Беггз Брилл', 'Факт КРД'])

In [ ]:
group_patterns = [["liquid_holdup"], ['fluid_flow.liquid_content']]
antipatterns = ['tpb', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns,plot_name='Истинное и расходное содержание жидкости',
                                 x_name = 'д.ед.', 
                                y_name = 'Глубина вдоль ствола скважины, м',
                                traces_names = [
                                               'Истинная доля жидкости в потоке ГЖС',
                                                'Расходная доля жидкости в потоке ГЖС'
                                               ])

In [ ]:
plotly_workflow.plot_by_patterns(result_df_p,  [['rho_oil_kgm3']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False, 
                                 plot_name='Плотность газонасыщенной нефти',
                                 x_name = 'Давление, бар', 
                                y_name = 'Плотность, кг/м3',
                                traces_names = [
                                               'Плотность, кг/м3',
                                               ]
                                )

In [ ]:
group_patterns = [["liberated_gas_sc_m3m3"],['dissolved_gas_sc_m3m3']]
antipatterns = ['tpb', 'number']
plotly_workflow.plot_by_patterns(result_df_p, group_patterns, antipatterns,
                                 print_all=False, swap_xy=False, reversed_y=False,
                                plot_name='Удельный объем выделившегося и растворенного газа',
                                x_name = 'Давление, бар', 
                                y_name = 'м3/м3',
                                traces_names = [
                                               'Удельный объем выделившегося газа, м3/м3',
                                    'Удельный объем растворенного газа, м3/м3'
                                               ])

In [ ]:
result_df_phi = result_df_p.copy()
result_df_phi = result_df_phi.set_index('fluid_flow.gas_fraction_d')
group_patterns = [["gas_fraction_real_d"]]
antipatterns = ['tpb', 'number']
plotly_workflow.plot_by_patterns(result_df_phi, group_patterns, antipatterns,
                                 print_all=False, swap_xy=False, reversed_y=False,
                                plot_name='Истинная доля газа от расходной',
                                x_name = 'Расходная доля газа, д.ед. (Без учета проскальзывания св. газа)', 
                                y_name = 'Истинная доля газа, д.ед. (с учетом проскальзывания св. газа)',
                                traces_names = [
                                               'Вязкость нефти, сПуаз',
                                               ])

Конструкция скважины

In [ ]:
trace1 = go.Scatter3d(
    x=simple_well.well_profile.x_displacement_m,
    y=simple_well.well_profile.x_displacement_m * 0,
    z=simple_well.well_profile.h_vert_m * (-1),  # перевод в альтитуду,
    marker=dict(
        size=4,
        colorscale='Viridis',
    ),
    name = 'Построенный профиль',
    )

trace2 = go.Scatter3d(
    x=simple_well.well_profile.get_x_displacement_m(simple_well.well_profile.h_mes_init_data_for_interpolation_m),
    y=simple_well.well_profile.get_x_displacement_m(simple_well.well_profile.h_mes_init_data_for_interpolation_m) * 0,
    z=simple_well.well_profile.h_vert_init_data_for_interpolation_m * (-1),
    mode = 'markers',
    marker = dict(
        size=10
    ),
    name = 'Исходные точки',
    )

data = [trace1, trace2]


layout = dict(title = '3D профиль простой скважины',  yaxis=dict(autorange='reversed'))
fig = dict(data=data, layout=layout)
iplot(fig, filename='simple-3d-scatter')

## Встроенный блок многовариативного анализа

Для данной адаптированной скважины при той же IPR меняем дебиты и рассчитываем несколько раз снизу-вверх

In [ ]:
results = []
for i in range(10, int(well_data['qliq_on_surface_m3day'] * 1.35), int((well_data['qliq_on_surface_m3day'] * 1.35 - 10) / 5) ):
    print(i)
    simple_well.data.clear_data()
    simple_well.qliq_on_surface_m3day = i
    simple_well.calc_all_from_down_to_up()
    this_result_df = simple_well.data.get_data_as_df()
    this_result_df = this_result_df.set_index('well.h_calculated_mes_m')
    results.append(this_result_df)
    
    this_result_df = this_result_df.add_prefix(f"qliq_m3day: {i}, ")
    try:
        result_df_to_plot = result_df_to_plot.join(this_result_df)
    except:
        result_df_to_plot = this_result_df.copy()

Сохраняем результаты

In [ ]:
result_df_analysis = None
for i in results:
    qliq_on_surface_m3day = i['well.qliq_on_surface_m3day'][0]
    this_df = pd.DataFrame({'well.p_bottomhole_bar': i['well.p_bottomhole_bar'][0],
                           'ipr.p_reservoir_bar': i['ipr.p_reservoir_bar'][0],
                           'well.p_calculated_bar': i['well.p_calculated_bar'][0],
                           'p_wellhead_bar': real_measurements['p_survey_mpa'][0]*10}, index = [qliq_on_surface_m3day])
    try:
        result_df_analysis = result_df_analysis.append(this_df)
    except:
        result_df_analysis = this_df.copy()

In [ ]:
patterns = [[x] for x in result_df_analysis.columns]
plotly_workflow.plot_by_patterns(result_df_analysis, patterns, [], plot_name='Определение максимального дебита скважины',
                                 print_all=False, swap_xy=False, reversed_y=False,
                                x_name='Дебит жидкости, м3/сут',
    y_name='Давление, бар',
    traces_names=['Давление на забое скважины, бар',
                 'Пластовое давление, бар',
                 'Устьевое давление, бар',
                 'Устевое давление по тех.режиму'])

Результаты по одной скважине из рассчитанных для последующего построения графиков

In [ ]:
result_df = results[0]

Результаты расчета для разных дебитов в одном DataFrame для последующего построения графиков

In [ ]:
result_df = result_df_to_plot

## Продолжение построения распределения свойств по глубине для выбранной скважины / нескольких скважин

### Название всех сохраненных параметров для выбора или не выбора их при построении графиков

In [ ]:
for i in result_df.columns:
    print(i)

In [ ]:
group_patterns = ["liquid_holdup"]
antipatterns = ['tpb', 'well_profile']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
group_patterns = [["p_calculated_bar"]]
antipatterns = ['mu', 'h_mes_survey', 'mpa','fl','ipr']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Расчет КРД',
                                 print_all=False,
                                x_name='Давление, бар',
    y_name='Глубина, м',
    traces_names=['КРД при Q=10 м3/сут', 'КРД при Q=18 м3/сут',
                 'КРД при Q=26 м3/сут','КРД при Q=34 м3/сут','КРД при Q=42 м3/сут', 'КРД при Q=50 м3/сут']
                                )